In [1]:
import os
import pandas as pd

from network_wrangler import ProjectCard

from update_registry import update_registry
from methods_io import read_project_cards
from methods_add_cards import add_cards_to_registry
from methods_add_cards import _make_available

## Input/Output

In [2]:
CARD_DIR = os.path.join(".", "projects")
REGISTRY_FILE = "registry.csv"
CONFIG_FILE = "registry_config.yml"

## Update Registry

In [3]:
update_registry(
        CONFIG_FILE, REGISTRY_FILE, "delete_me.csv", CARD_DIR, False,
    )

KeyError: 'category'